# Modeling - XGBoost

In [1]:
import sys
import os
sys.path.append('../../src')


import numpy as np
import pickle
from models.lightgbm_model import LightGBModel
from tuning import random_search_tuning
from utils import  generate_combinations, generate_dataset_split, save_combination
from itertools import combinations
from lightgbm import LGBMClassifier



c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters for tuning

In [2]:
parameters = {
    'num_leaves':  np.arange(20, 50, 10),
    'learning_rate': np.arange(0.001, 0.1, 0.005),
    'max_depth': np.arange(2, 8),
    'min_data_in_leaf': np.arange(10, 30, 5),
    'min_gain_to_split': np.arange(0, 0.05, 0.01),
    'n_estimators': np.arange(50, 150, 10),
    'colsample_bytree': np.arange(0.6, 1.0, 0.1),
    'early_stopping_rounds': [5],
    'objective': ['multiclassova'], 
    'num_class': [3],
    'metric':['auc_mu'],
    'device': ['gpu'],
}

## Modeling

In [3]:


cities_siglas = {
    "A": "Porto Alegre",
    "B": "Marabá",
    "C": "Brasília",
    "D": "Belo Horizonte",
    "E": "Juazeiro do Norte",
    "F": "Recife"
}

cities = ["Porto Alegre", "Marabá", "Brasília", "Belo Horizonte", "Juazeiro do Norte", "Recife"]

polos_sigla = ['A', 'B', 'C', 'D', 'E', 'F']
polos = [cities_siglas[s] for s in polos_sigla]

In [4]:
splits = []
for i in range(0, len(polos_sigla)):
    splits.append(generate_combinations(polos[:i] + polos[i+1:], 3, 2))

In [5]:

models = []
save_combination(f'split3_2/{cities[i]}', "multiclass", splits[0])
for idx, combination in enumerate(splits[0], start=0):
    print(f"Combinação {idx}: {combination} training")
    X_train, y_train = generate_dataset_split(combination[0], "multiclass")
    X_val, y_val = generate_dataset_split(combination[1], "multiclass")
    clf = LGBMClassifier()
    model = random_search_tuning(clf, parameters)
    model = model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    print(model.best_estimator_)
    models.append(model)

for j in range(0, len(models)):
    pickle.dump(models[j], open(f"../../data/models/multiclass/split3_2/{cities[0]}/xgb_{j}.sav", 'wb'))
    


Combinação 0: [['Marabá', 'Brasília', 'Belo Horizonte'], ['Juazeiro do Norte', 'Recife']] training


c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Info] Number of positive: 265926, number of negative: 48446
[LightGBM] [Info] Number of positive: 12137, number of negative: 302235
[LightGBM] [Info] Number of positive: 36309, number of negative: 278063
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 16646
[LightGBM] [Info] Number of data points in the train set: 314372, number of used f

KeyboardInterrupt: 